In [1]:
from utils import *
import json
import random
import pandas as pd
import numpy as np

class Args:
    data_frac = 0.3
    min_user_freq = 10
    min_book_freq = 10
    max_user_freq = 200
    train_frac = 0.95
    his_len = 100
    n_neg = 10


args = Args()


In [4]:
test_samples = pkl_read('data/test_samples.pkl')

{'his': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, '0099270234', '0425158594', '0451194861', '0451159330',
        '0441014100', '0060573775', '0060510862', '031218087X',
        '0879979798', '038072345X', '0140390464', '0385479565',
        '0451191137', '0099427869', '034538475X', '0345389409',
        '0451194004', '0385418485', '0553280945', '0312963009',
        '034071820X', '0141040343', '0060098902', '0671042858',
        '0345487133', '0811825558', '0060191988', '0440224071',
        '0141439688', '0440225345', '0553572997', '0140434135',
        '0060755334', '0684843323', '0446523364', '0684843749',
        '0671028448', '0399144986', '0671024256', '0679722319',
        '0944092691', '0679601724', '0140042520', '0553208845',
        '031286504X', '0307021343', '0743211383', '0399145761',
        '0345444884', '0375503803', '0345441036', '0316013684',
        '0060976241', '0786114398', '0452280028', '0684836270',
        '0340770732'